<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/Insight-Sogang-Univ/insight-14th/blob/main/advanced/template/session05/assignment_languagemodel.ipynb" target="_parent">
      <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
    </a>
  </td>
</table>


# 🔍 이론 문제

### 1. SLM(통계적 언어모델)의 한계에 대해 설명하세요.

**[1번 문제 정답]**

***희소 문제를 가집니다. 희소 문제란, 모델이 가지고 있는 데이터가 부족해서 언어를 정확하게 모델링하지 못하는 문제를 뜻합니다.*** \
[정답]


### 2. BERT에 대한 설명이 아닌 것은?

a) 사전 학습 모델

b) 양방향 문맥 이해

c) 트랜스포머의 디코더를 쌓아 올린 구조

d) 이후 RoBERTa와 ALBERT로 발전

**[2번 문제 정답]**

***c*** \
[정답]

### 3. BERT와 GPT의 학습 방식 차이를 간단히 설명하세요.

**[3번 문제 정답]**

***BERT는 양방향으로 문장의 문맥을 학습하고, GPT는 앞 부분 단어만 보고 단방향으로 다음 단어를 생성합니다.*** \
[정답]

### 4. LLM이 지닌 한계와 RAG의 필요성에 대해 간단히 서술하세요.

**[4번 문제 정답]**

***모르는 정보로 인하여 LLM이 사실과 다른 내용을 만들어내는 Hallucination 문제가 있습니다. RAG는 검색을 통해 실제 데이터에 기반한 답변을 생성할 수 있도록 하여 이러한 환각 문제를 보완합니다.*** \
[정답]

### 5. LLM을 경량화한 모델인 sLM을 만들기 위해 사용되는 대표적인 기술 3가지를 적으세요.(단답형)

**[5번 문제 정답]**

***가지치기, 양자화, 지식 증류*** \
[정답]

# ✍ 언어 모델 과제

### 🧮 GPU 사용하기

In [ ]:
import torch
import tensorflow as tf

# PyTorch GPU 상태
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ [PyTorch] Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")


# TensorFlow GPU 상태
print("\n✅ [TensorFlow] GPU Devices:")
print(tf.config.list_physical_devices('GPU'))

✅ [PyTorch] Using device: cuda
GPU name: Tesla T4

✅ [TensorFlow] GPU Devices:
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# 🔍 1. BERT

**BERT (Bidirectional Encoder Representations from Transformers)**
- 문장 분류(스팸 메일 탐지), 질의응답 시스템(챗봇, 검색 엔진), 번역(다국어 지원), 텍스트 요약(뉴스 요약) 등 다양한 NLP 작업에 사용
- 문맥을 양방향으로 이해하여 텍스트의 의미를 정밀하게 파악하며, 사전 학습된 모델을 기반으로 빠르게 응용 가능

### 1-1. 모델과 tokenizer 초기화
- tokenizer를 통해 문장을 토큰으로 나누고, 이를 정수 인덱스로 변환하여 모델이 이해할 수 있도록 함

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# BERT 모델과 tokenizer 초기화
tokenizer = BertTokenizer.from_pretrained('klue/bert-base') # base : 모델크기, (uncased : 소문자 학습)
model = BertForMaskedLM.from_pretrained('klue/bert-base') # base : 모델크기, (uncased : 소문자 학습)
# tokenizer : raw text를 개별 토큰으로 분리(Wordpiece). 토크나이저 사전에 따라 고유한 정수 인덱스(고정값)를 매핑함.

# tokenizer를 통해 생성된 정수 인덱스 : 텍스트를 모델이 읽을 수 있도록 숫자화
# Embedding vector : 단어의 의미적, 문맥적 특성을 모델링

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# 테스트할 문장
text = "[CLS] 열여덟, 우리는 서로의 이름을 처음 불렀다. 그리고 스물 하나, 우린 [MASK]을 했다.[SEP]"

# 문장을 토큰으로 변환
tokenized_text = tokenizer.tokenize(text) # text(문장)을 토큰으로 변환해 tokenized_text에 저장합시다!
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text) # 토큰을 index로 변환하는 코드!

print(tokenized_text) # 토큰이 출력됩니다.
print(indexed_tokens) # 인덱스가 출력됩니다.


['[CLS]', '열', '##여', '##덟', ',', '우리', '##는', '서로', '##의', '이름', '##을', '처음', '불렀', '##다', '.', '그리고', '스물', '하나', ',', '우린', '[MASK]', '을', '했', '##다', '.', '[SEP]']
[2, 1432, 2173, 3542, 16, 3616, 2259, 4084, 2079, 3934, 2069, 3790, 6895, 2062, 18, 3673, 10514, 3657, 16, 8983, 4, 1498, 1902, 2062, 18, 3]


### 1-2. 입력 텍스트 준비 및 토큰화
- [MASK] 토큰을 삽입한 문장을 설정하고, `tokenizer.tokenize()`로 텍스트 토큰화
- [MASK]를 활용하여 문맥 속에서 특정 단어를 추론하는 상황을 만들고, 이를 모델이 학습한 패턴과 비교하도록 함


### 1-3. MASK된 위치 확인
- [MASK] 토큰의 위치를 탐지해 해당 위치에서 모델이 단어를 예측하도록 지정합니다.


In [ ]:
# 마스킹된 위치 찾기
masked_index = tokenized_text.index("[MASK]") # 토큰화된 결과물에서 index 메서드를 통해 [MASK]의 위치를 확인해봅시다!
print(masked_index)

20


### 1-4. 정수화된 토큰 인덱스를 tensor로 변환 후 모델 예측 실행
- 모델은 **텐서 데이터 구조**를 필요로 하므로, **입력값을 변환**하여 적합한 형식으로 만듦.
   - `torch.tensor(입력값 리스트 또는 입력값 ndarray)`를 통해 tensor 구조로 변환 가능!
- **역전파를 비활성화**하고, 모델이 **각 토큰에 대해 가능한 모든 단어 점수 출력**

In [ ]:
# 토큰화된 텍스트 -> 정수인덱스(ID) -> Pytorch 텐서
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text) # 토큰을 index로 변환하는 코드!
tokens_tensor = torch.tensor([indexed_tokens]) # 입력값(토큰 인덱스 리스트)을 tensor 구조로 변환해봅시다!

# 모델에 토큰 텐서를 전달하고 예측 실행
with torch.no_grad(): # 가중치 업데이트 없으므로 자동미분연산은 끔
    outputs = model(tokens_tensor) # 예측. 모델은 각 토큰위치에 대한 예측 결과를 반환.
    predictions = outputs[0] # logits (각 토큰위치에서 가능한 모든 토큰들의 원시점수)
print(predictions)

tensor([[[ -6.0337,   4.5526,  -5.6312,  ...,  -7.3959,  -7.4220,  -5.8337],
         [ -6.1762,   4.7585,  -7.3973,  ...,  -7.6209, -12.2124,  -6.0998],
         [ -7.4194,   3.9148,  -6.1517,  ...,  -6.8162,  -9.5421,  -3.0473],
         ...,
         [ -7.3761,   8.6972,  -5.4904,  ...,  -8.7202,  -9.1403,  -5.8566],
         [ -5.6347,  10.3884,  -4.3374,  ...,  -8.6900,  -7.4171,  -3.5043],
         [ -5.6652,  10.3407,  -4.4099,  ...,  -8.7521,  -7.4286,  -3.6681]]])


### 1-5. MASK된 토큰 예측
- [MASK] 위치에서 가장 높은 점수를 받은 토큰의 인덱스를 추출하고, 이를 단어로 변환

In [ ]:
# 예측된 토큰 확인
predicted_index = torch.argmax(predictions[0, masked_index]).item() # masked_index에 들어갈 것으로 가장 확률이 높은 인덱스
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

# 예측된 토큰으로 마스크 채우기
tokenized_text[masked_index] = predicted_token
# 토큰화된 텍스트를 다시 문자열로 변환
filled_text = tokenizer.convert_tokens_to_string(tokenized_text[1:-1])

print("Original:", text)
print("Masked:", tokenized_text)
print("Predicted token:", predicted_token) ; print()

print("Filled sentence:", filled_text)

Original: [CLS] 열여덟, 우리는 서로의 이름을 처음 불렀다. 그리고 스물 하나, 우린 [MASK]을 했다.[SEP]
Masked: ['[CLS]', '열', '##여', '##덟', ',', '우리', '##는', '서로', '##의', '이름', '##을', '처음', '불렀', '##다', '.', '그리고', '스물', '하나', ',', '우린', '사랑', '을', '했', '##다', '.', '[SEP]']
Predicted token: 사랑

Filled sentence: 열여덟 , 우리는 서로의 이름을 처음 불렀다 . 그리고 스물 하나 , 우린 사랑 을 했다 .



### 1-6. MASK된 문장 복원
- 예측된 토큰으로 [MASK]를 대체하여 완성된 문장 생성

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# BERT 모델과 토크나이저 초기화
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')
model = BertForMaskedLM.from_pretrained('klue/bert-base')

def fill_mask(input_text):
    # 텍스트를 토큰으로 변환
    tokenized_text = tokenizer.tokenize(input_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

    # 마스킹된 위치 찾기
    masked_index = tokenized_text.index("[MASK]") # 토큰화된 텍스트에서 마스킹된 위치를 찾아봅시다!

    # 토큰화된 텍스트 -> 정수인덱스(ID) -> Pytorch 텐서
    tokens_tensor = torch.tensor([indexed_tokens])

    # 모델에 토큰 텐서를 전달하고 예측 수행
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    # 예측된 토큰 확인
    predicted_index = torch.argmax(predictions[0, masked_index]).item() # 위 코드 참고! masked_index에 들어갈 것으로 가장 확률이 높은 인덱스
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

    # 마스크를 채운 문장 반환
    tokenized_text[masked_index] = predicted_token # 토큰화된 텍스트의 마스킹된 위치(인덱스 사용)를 예측된 토큰으로 채워봅시다!
    return tokenizer.convert_tokens_to_string(tokenized_text[1:-1])

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


예시 문장 만들어서 MLM을 사용해 보세요!

In [ ]:
# 예시 1: "[CLS] 열여덟, 우리는 서로의 이름을 처음 불렀다. 그리고 스물 하나, 우린 [MASK]을 했다.[SEP]"
# 예시 2: "[CLS] 윤해민은 INSGIHT 학회를 정말 사랑한다. 그는 INSIGHT 학회를 위해 [MASK]를 했다.[SEP]"
# 이 예시를 참고해서 [CLS](맨 앞), [MASK](채울 부분), [SEP](맨 뒤)를 추가해서 여러분만의 예시 문장 만들어주세요
input_text = "[CLS] 개굴개굴개구리 [MASK]를 한다.[SEP]"
filled_text = fill_mask(input_text)
print("Filled sentence:", filled_text)

Filled sentence: 개굴개굴개구리 놀이 를 한다 .


# 🔍 2. GPT, RAG, Langchain

## 2-1 라이브러리 설치

langchain 관련 라이브러리와, pdf 문서를 읽을수 있는 pypdf 라이브러리를 설치할게요!

In [ ]:
# ⬇️ Colab에서 한 번만 실행
%pip install -qU "langchain>=0.3" langchain-community langchain-openai langgraph chromadb pypdf tiktoken


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.7/93.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.5/82.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 111.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 39.0 MB/s eta 0

## 2-2. 키값 설정

## 2-3 pdf 문서 선택 (로드)


여러분들이 활용하고 싶은 아무 pdf 문서를 선택해보세요!

In [ ]:
from google.colab import files
import os, re, unicodedata

os.makedirs("/content/pdfs", exist_ok=True)
print("📥 업로드할 PDF 파일을 선택하세요 (여러 개 가능).")
uploaded = files.upload()

name_map = {}
for i, (orig_name, data) in enumerate(uploaded.items(), 1):
    nf = unicodedata.normalize("NFC", orig_name)
    base, ext = os.path.splitext(nf)
    # 한글/영문/숫자/밑줄/대시/점만 남기고 나머지는 _
    safe_base = re.sub(r"[^\w\u3131-\u318E\uAC00-\uD7A3.\-]+", "_", base).strip("._'‘’“”")
    # 파일명 너무 길면 잘라냄 (최대 ~140자 정도로)
    safe_base = safe_base[:120] if len(safe_base) > 120 else safe_base
    new_name = f"doc_{i}_{safe_base}{ext.lower()}"
    with open(os.path.join("/content/pdfs", new_name), "wb") as f:
        f.write(data)
    name_map[new_name] = orig_name

print("✅ 저장 완료 (Colab 파일명 → 원본명 매핑):")
for k, v in name_map.items():
    print(f"  {k}  <=  {v}")


📥 업로드할 PDF 파일을 선택하세요 (여러 개 가능).


Saving Algorithmic+Bias+in+Recommendation+Systems+and+Its+Social+Impact+on+User+Behavior.pdf to Algorithmic+Bias+in+Recommendation+Systems+and+Its+Social+Impact+on+User+Behavior.pdf
✅ 저장 완료 (Colab 파일명 → 원본명 매핑):
  doc_1_Algorithmic_Bias_in_Recommendation_Systems_and_Its_Social_Impact_on_User_Behavior.pdf  <=  Algorithmic+Bias+in+Recommendation+Systems+and+Its+Social+Impact+on+User+Behavior.pdf


## 2-4 문서 분할 후 벡터 DB에 저장
로딩한 문서를 분할 할 때,
- chunk_size는 `900`
- chunk_overlap은 `150`
- separators(구분자)는 `["\n\n", "\n", " ", ""]`
으로 설정하여 청킹해주세요!

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
import glob

# 1) PDF → Documents
pdf_paths = sorted(glob.glob("/content/pdfs/*.pdf"))
docs = []
for p in pdf_paths:
    try:
        loader = PyPDFLoader(p)
        docs.extend(loader.load())
    except Exception as e:
        print(f"⚠️ 로딩 실패: {p} -> {e}")

print(f"총 문서 조각(페이지 기준): {len(docs)}")

# 2) 문서 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 800, # 청크 사이즈: 문서를 몇 개의 토큰 단위로 나눌 것인지를 정합니다.
    chunk_overlap= 150, # 청크 오버랩: 분할된 끝 부분에서 맥락이 이어질 수 있도록 일부를 겹쳐서(overlap) 분할하는 것이 일반적입니다.
    separators= ["\n\n", "\n", " ", ""], # 구분자: 엔터, 공백 기준으로 나눔
)
chunks = text_splitter.split_documents(docs)
print(f"총 청크 수: {len(chunks)}")

# 3) 벡터스토어 (영속 디렉토리 쓰면 런타임 재시작해도 유지)
persist_dir = "/content/chroma_pdf_db"
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 비용 절약형
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=persist_dir
)
retriever = vectordb.as_retriever(search_kwargs={"k": 5})
print("✅ 인덱싱 완료")


총 문서 조각(페이지 기준): 14
총 청크 수: 85
✅ 인덱싱 완료


## 2-5 RA<U>Generate</U> : 생성 부분 선언

PROMPT 부분은 여러분들이 원하시는대로 입력해도 좋습니다!

🤥 영어로 입력하면 모델이 더 잘 이해한대요

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

PROMPT = ChatPromptTemplate.from_template(
"""다음 컨텍스트(업로드한 PDF에서 추출됨)만을 바탕으로 질문에 한국어로 답하라.
불확실하면 모른다고 말하라. 핵심 근거를 요약하고, 각 근거 옆에 출처(파일/페이지)를 표시하라.

반드시 지켜야 할 규칙:
1) **컨텍스트에서 명시적으로 제공된 정보만 사용**하고, 외부 지식으로 보충하지 마십시오.
2) 답변의 모든 주장에는 **핵심 근거 요약과 출처(파일명/페이지)**를 함께 표시하십시오.
3) 만약 컨텍스트에 답변 근거가 불충분하거나 모호하다면,
   **추론하지 말고 "컨텍스트만으로는 알 수 없음"**이라고 답하십시오.
4) 논문의 주제(알고리즘, 미디어, 추천 시스템, 플랫폼 구조 등)에 맞춰
   **학술적으로 정확한 용어**를 사용하십시오.
5) 답변은 **한국어**로 작성하십시오.

[질문]
{question}

[컨텍스트]
{context}

[답변 요약]
- 질문에 대한 핵심 답변 2~4줄

[근거 및 출처]
- 근거1: 요약 내용 (파일명, p.XX)
- 근거2: 요약 내용 (파일명, p.XX)
- …

[판단]
- 컨텍스트 정보만으로 답변 가능 여부: 가능 / 불가능(불충분)
"""
)

rag_chain = (PROMPT | llm | StrOutputParser())


## 2-6 RAG 파이프라인 langgraph로 구성

벡터db에서 검색하는 것이 <U>Retrieve</U>AG이고,

검색 결과를 생성 단계에 던져주는 것이 R<U>Augment</U>G입니다!

In [ ]:
from typing import TypedDict, List
from langchain_core.documents import Document
from langgraph.graph import StateGraph, END

# format_docs의 역할은 문서를 포맷팅해서 모델에게 제공할 'context'를 정리하는 역할입니다!
def format_docs(docs):
    # 파일명/페이지를 메타데이터에 담아두는 PyPDFLoader 기본값 사용
    lines = []
    for i, d in enumerate(docs, 1):
        src = d.metadata.get("source", "")
        page = d.metadata.get("page", None)
        tag = f"{os.path.basename(src)}"
        if page is not None:
            tag += f" p.{page+1}"
        lines.append(f"[{i}] {tag}\n{d.page_content}")
    return "\n\n".join(lines)

# 그래프 선언
class QAState(TypedDict):
    question: str
    retrieved: List[Document]
    answer: str

def node_retrieve(state: QAState):
    q = state["question"]
    topk = retriever.invoke(q)
    return {"retrieved": topk}

def node_generate(state: QAState):
    q = state["question"]
    ctx = format_docs(state["retrieved"])
    ans = rag_chain.invoke({"question": q, "context": ctx})
    return {"answer": ans}

# --------- 그래프 구성 ---------
# 노드끼리 연결하기
workflow = StateGraph(QAState)
workflow.add_node("retrieve", node_retrieve) # 어떤 노드가 추가되어야 할까요?
workflow.add_node("generate", node_generate) # 어떤 노드가 추가되어야 할까요?

workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate") # 어떤 노드와 어떤 노드가 연결되어야 할까요?
workflow.add_edge("generate", END)

app = workflow.compile()
print("✅ LangGraph 컴파일 완료")


✅ LangGraph 컴파일 완료


## 2-7 실제 구동해보기

question에 여러분들이 입력하고 싶은 질문을 넣어보세요!

RAG 과제는 이렇게 간단하게 끝내겠습니다😁

In [ ]:
question = "아도르노와 호르크하이머의 ‘문화산업’ 비판이 현대 뉴미디어 환경에서도 여전히 유효한가? 특히 **알고리즘 기반 추천 시스템**, **플랫폼 구조**, **사용자 데이터 처리 방식** 등과 연결하여 컨텍스트 속 논문 내용만을 근거로 판단하라."
final = app.invoke({"question": question})
print("▼ 답변")
print(final["answer"])


▼ 답변
아도르노와 호르크하이머의 ‘문화산업’ 비판은 현대 뉴미디어 환경에서도 여전히 유효하다. 특히 알고리즘 기반 추천 시스템은 사용자에게 필터 버블을 형성하고 콘텐츠 다양성을 감소시키며 사회적 불평등을 강화하는 경향이 있다. 이러한 문제는 추천 시스템의 설계와 운영에서 기술적, 윤리적, 정책적 개입이 필요함을 시사한다.

[근거 및 출처]
- 근거1: 알고리즘 편향이 필터 버블, 콘텐츠 다양성 감소, 사회적 불평등 강화와 관련이 있다 (doc_1, p.1)
- 근거2: 추천 시스템의 모델 편향이 인기 콘텐츠를 선호하게 하여 다양성을 제한하고, 피드백 루프가 이러한 문제를 심화시킨다 (doc_1, p.11)
- 근거3: 사회적으로 책임 있는 추천 시스템을 설계하기 위해 기술적, 윤리적, 정책적 접근이 통합되어야 한다 (doc_1, p.5)

[판단]
- 컨텍스트 정보만으로 답변 가능 여부: 가능


In [ ]:
question = "이 논문에 대해서 요약해줘"
final = app.invoke({"question": question})
print("▼ 답변")
print(final["answer"])


▼ 답변
이 논문은 추천 시스템에서의 알고리즘 편향의 원인, 영향 및 완화 전략을 포괄적으로 연구하고 있다. 연구는 이론적 분석과 사례 연구를 결합하여 알고리즘 편향이 사용자 행동에 미치는 영향을 평가하고, 이를 해결하기 위한 실질적인 권고사항을 제시한다. 또한, 데이터 편향, 모델 편향, 피드백 루프 등 다양한 차원에서 알고리즘 편향을 분석하고, 향후 연구 방향을 제시한다.

[근거 및 출처]
- 근거1: 이 연구는 알고리즘 편향의 원인, 영향 및 완화 전략을 포괄적으로 다룬다 (doc_1_Algorithmic_Bias_in_Recommendation_Systems_and_Its_Social_Impact_on_User_Behavior.pdf, p.11).
- 근거2: 연구는 이론적 분석과 사례 연구를 결합하여 알고리즘 편향이 사용자 행동에 미치는 영향을 평가한다 (doc_1_Algorithmic_Bias_in_Recommendation_Systems_and_Its_Social_Impact_on_User_Behavior.pdf, p.5).
- 근거3: 데이터 편향, 모델 편향, 피드백 루프 등 다양한 차원에서 알고리즘 편향을 분석하고 향후 연구 방향을 제시한다 (doc_1_Algorithmic_Bias_in_Recommendation_Systems_and_Its_Social_Impact_on_User_Behavior.pdf, p.11).

[판단]
- 컨텍스트 정보만으로 답변 가능 여부: 가능


In [ ]:
question = "이 논문에 있는 주요 통계 자료를 정리해서 얘기해 줘"
final = app.invoke({"question": question})
print("▼ 답변")
print(final["answer"])


▼ 답변
[답변 요약]
이 논문에서는 알고리즘 편향의 원인과 그 사회적 영향을 다루고 있으며, 추천 시스템에서의 알고리즘 편향이 사용자 행동에 미치는 영향을 분석하고 있습니다. 주요 통계 자료는 Netflix, YouTube, Amazon의 사례 연구를 통해 데이터 편향, 모델 편향, 피드백 루프의 차원에서 제시됩니다.

[근거 및 출처]
- 근거1: 알고리즘 편향의 원인과 영향을 다루며, 추천 시스템에서의 편향을 평가하는 연구 방법론이 설명됨 (doc_1_Algorithmic_Bias_in_Recommendation_Systems_and_Its_Social_Impact_on_User_Behavior.pdf, p.3)
- 근거2: 데이터 편향, 모델 편향, 피드백 루프의 차원에서 알고리즘 편향의 결과가 논의됨 (doc_1_Algorithmic_Bias_in_Recommendation_Systems_and_Its_Social_Impact_on_User_Behavior.pdf, p.8)

[판단]
- 컨텍스트 정보만으로 답변 가능 여부: 가능


# 🔍 3. sLM

Copyright 2024 Google LLC.

## 3-1. sLM(Gemma) 로드
### sLM(small Language Model)의 대표 모델인 Gemma를 로드 해봅시다!

### (1) 사전 준비 사항
### 1. HuggingFace 로그인

https://huggingface.co/

이 링크로 들어가서 HuggingFace에 로그인합니다.(계정이 없다면 회원가입 해주세요!)

### 2. 토큰 발급받기

https://huggingface.co/settings/tokens

- 이 링크로 들어가서 'Create new token'을 클릭
- Token type: 'Read'로 해주세요.
- Token name: 아무거나 상관 없습니다.
- (⭐ 매우 중요!) 토큰 만드시고 **반드시 토큰 문자열 (hf_... 로 시작함)을 복사해주세요!**  
한 번만 볼 수 있습니다!

### 3. google colab에서 토큰 등록하기
- 왼쪽에 열쇠 모양 버튼(보안 비밀)을 눌러주세요.
- 이름: colab-gemma
- 값: 복사해둔 토큰 문자열(hf_...로 시작함)
- 노트북 액세스 : 스위치 키기(파란색)

### 4. gemma-2b-it 모델 사용 약관 동의

https://huggingface.co/google/gemma-2b-it

- (⭐ 매우 중요!) 반드시 2번 단계에서 토큰을 발급받았던 바로 그 Hugging Face 계정으로 로그인된 상태에서 위 링크로 들어가야 합니다.
- 링크로 들어가서 'Access Gemma on Hugging Face' 부분의 약관을 읽고 동의 버튼을 클릭해 주세요.
- "You have been granted access to this model"이라는 메시지가 뜨면 성공입니다.

### (2) 필요한 툴 설치

In [ ]:
!pip install -U bitsandbytes transformers accelerate
!pip install --upgrade -q huggingface_hub peft \
  datasets trl

  Using cached bitsandbytes-0.48.2-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.48.2-py3-none-manylinux_2_24_x86_64.whl (59.4 MB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.43.1
    Uninstalling bitsandbytes-0.43.1:
      Successfully uninstalled bitsandbytes-0.43.1


In [ ]:
# HuggingFace에 로그인합니다.

from huggingface_hub import login
from google.colab import userdata

# 따음표 안에 본인의 보안 비밀 키의 이름을 적습니다.(여기선 colab-gemma)
try:
    login(token=userdata.get("colab-gemma"))
except Exception as e:
    print(f"Error logging in to Hugging Face: {e}")
    print("Please make sure you have added your Hugging Face token to Colab secrets with the name 'colab-gemma'")

In [ ]:
# 모델을 로드합니다.
import torch
import bitsandbytes # Explicitly import bitsandbytes here
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 계산 부하를 줄이고 추론 속도를 높이기 위해 '양자화'합니다.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# sLM(gemma)를 Q&A, 요약 등에 활용하기 위해 '지시 튜닝(Instruction-Tuned) 버전의 모델을 사용합니다.
# "google/gemma-2b-it" 모델을 사용합니다.
model_id = "google/gemma-2b-it"

# 토크나이저를 로드합니다.
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 모델을 로드합니다.
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config, # 양자화(quantization)
    device_map={"":"cuda:0"}
)

# GPU를 사용하도록 설정합니다.
device = "cuda"

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

## 3-2. sLM 활용
- sLM은 LLM을 경량화한 모델이기 때문에 언어 모델의 기능인 질의응답(추론), 요약, 분류(감정분석) 등이 가능합니다.
- **속도는 비교적 빠르지만, 성능이 그닥 좋지 않다는 것을 확인하실 수 있습니다.**

### Gemma 활용 예시 (Common Use Cases)


### (1) 질의응답(Reasoning)

#### 원하는 질문을 한 번 넣어보세요!

In [ ]:
print("--- 예시 1: 질의응답 (Reasoning) ---")

# content : '~~' 따음표 안에 질문을 써주시면 됩니다.
# 하고 싶은 질문 아무거나 넣어보세요!

# "role": "user" 입니다.

chat = [
    # Gemma 모델의 채팅 형식에 맞춰 'role'과 'content'를 지정합니다.
    { "role": "user", "content": "금융권에서 AI가 어떻게 활용되고 있는지 알려줘" }
]

# 1. 딕셔너리 리스트('chat')를 Gemma 모델이 이해할 수 있는 공식 프롬프트 문자열로 변환합니다.
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

# 2. 변환된 프롬프트 문자열을 모델이 입력받을 수 있는 토큰 ID 텐서로 변환하고, GPU('device')로 보냅니다.
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# 3. 토큰화된 입력을 모델에 전달하여 텍스트 ""생성""을 시작합니다 (최대 256개의 새 토큰 생성)
# 생성을 위해 'generate' 함수를 사용합니다.
outputs = model.generate(**inputs, max_new_tokens=256) # **inputs에서 **는 빈칸이 아닙니다.

# 4. 'outputs'에는 '입력'과 '답변'이 모두 포함되어 있습니다.
# 따라서 원본 입력 토큰의 길이(입력, 'inputs.input_ids.shape[1]')를 기준으로, 그 이후에 '새로 생성된' 토큰(답변)만 추출합니다.
new_tokens = outputs[0][inputs.input_ids.shape[1]:]

# 5. 추출된 토큰 ID 리스트를 다시 사람이 읽을 수 있는 문자열로 ""디코딩""합니다.
# 디코딩을 위해 'decode' 함수를 사용합니다.
text = tokenizer.decode(new_tokens, skip_special_tokens=True)

# 6. 최종 생성된 답변 텍스트를 화면에 출력합니다.
print(text)

--- 예시 1: 질의응답 (Reasoning) ---
금융권에서 AI는 다음과 같은 역할을 수행할 수 있습니다.

**1. 데이터 분석 및 예측**

* 금융 기관은 AI를 사용하여 수익과 위험을 예측하기 위해 데이터를 분석합니다.
* AI는 기업의 운영, 투자 및 재무 상태를 분석하여 미래 수익을 예측할 수 있습니다.
* AI는 또한 기업의 위험을 예측하고 대응하기 위해 사용될 수 있습니다.

**2. 자기회귀 모델 개발**

* AI는 자기회귀 모델을 개발하여 기업의 수익과 위험을 예측할 수 있습니다.
* 자기회귀 모델은 과거 데이터를 기반으로 미래 데이터를 예측하는 데 사용됩니다.
* AI는 자기회귀 모델을 사용하여 기업의 수익과 위험을 최적화할 수 있습니다.

**3. 투자 분석**

* AI는 투자자에게 최적의 투자 기회를 찾기 위해 투자 분석을 수행할 수 있습니다.
* AI는 수익과 위험을 고려한 투자 기회를 선


### (2) 요약 (Summarization)

#### 원하는 글을 넣어서 요약해 보세요!

In [ ]:
print("\n--- 예시 2: 요약 (Summarization) ---")

# text_to_summarize = """~~~""" 따음표 안에 요약하고 싶은 글을 넣어주시면 됩니다.""
# 요약하고 싶은 글 아무거나 넣어보세요!

text_to_summarize = """
2022년 봄 회사 MBA 장기연수 프로그램에 선발된 이후 2025년 여름 졸업에 이르기까지 햇수로 4년 동안 나의 MBA 생활을 지탱하는 하나의 큰 축은 가족, 그리고 다른 한 축은 솔직히 이야기하면 ChatGPT였다. 2022년부터 2025년까지가 ChatGPT와 같은 생성형 AI의 태동기라고 말할 수는 없지만 AI 기술 진보 측면에서 패러다임이 변화하는 전환기라는 점은 분명히 느꼈다.



2022년 5월에 MBA 프로그램에 선발된 뒤 한창 시험을 준비하는 동안 ChatGPT에 대해서는 솔직히 거의 들어보지 못했다. MBA 프로그램에 지원하기 위해서는 GMAT이라는 시험을 봐야하는데 이건 AI가 문제를 대신 풀어줄 수도 없고 오로지 내가 스스로 공부해서 결과를 내야하기 때문에 오로지 인간인 나의 끈기와 지능만 사용되었다. ChatGPT를 처음 알게된 것은 GMAT 시험을 마무리짓고 본격적으로 미국 MBA 프로그램에 원서를 접수하기 시작한 2022년 초겨울 무렵이었다. 어느 한 곳이라도 합격했으면 좋겠다는 마음으로 열 곳 이상의 학교에 원서를 제출했는데, 학교마다 요구하는 서류도 조금씩 달랐고 서류와 함께 제출해야 하는 에세이 질문도 조금씩 달랐다. 대학을 지나 취업뽀개기 관문을 거쳐 직장생활을 시작한 사람이라면 누구나 공감하겠지만, 입사 원서를 지원할 때도 회사마다 물어보는 에세이 문항이 크게 다르지 않지만 똑같은 내용을 복사-붙여넣기 할 수는 없는 일이다. 내가 지원하는 기업에 맞게, 내가 지원하는 학교에 맞게 조금씩 내용을 바꾸고 다듬고 또 새로운 에세이 내용을 늘 고민해야 했다.



그렇게 2022년 12월에 지원 원서를 제출하고 난 뒤 몇 주 뒤에 다가올 인터뷰 면접을 준비하면서 ChatGPT의 이름을 처음 알게 되었다. (지금 생각하면 남들보다 비교적 늦게 알았던 셈이다.) ChatGPT는 일종의 한글-영어 번역기로 다가왔다. 영어 인터뷰에서 해야 할 답변을 미리 가볍게 영어로 정리해두는 편이 나을 것 같아서 예상 질문에 대한 답안을 준비하는데 나는 우선 한글로 내가 이야기 할 답안을 작성한 뒤, 그걸 ChatGPT에 영어로 번역해달라고 한 후, 마지막으로 문어체의 영어를 구어체로 다듬는 과정을 반복했다. 물론 그 당시에도 ChatGPT가 훨씬 다양한 기능을 제공했다는 걸 알았다면, 예상 질문을 입력한 뒤 “영어 인터뷰용 답안을 작성해줘.” 라고 프롬프트를 입력해서 손쉽게 답을 얻을 수 있었겠지만 그때만 해도 나는 ChatGPT를 단순한 번역 기능을 제공하는 도구로서 이해했다. 나와 함께 MBA를 준비하는 이들도 크게 다르지 않았다. 우리는 네이버 파파고, 구글 번역, 다음 카카오 번역 등 국내외에서 서비스를 제공하는 다른 번역 사이트와 ChatGPT를 동등한 선 위에서 비교했고, 어떤 사이트가 영어 인터뷰에 가장 자연스러운 영어 번역을 제공하는지 등의 정보를 나누었다. 어떤 번역 사이트는 인터뷰 번역을 자연스럽게 해준다고 하고, 어떤 번역 사이트는 서술형 에세이를 매끄럽게 번역해준다는 식의 이야기가 오갔다.



2023년 봄 MBA 프로그램에 합격한 뒤 학교에서 정식으로 MBA과정이 시작되자 가장 놀랐던 것 중 하나는 생각보다 학교에서 ChatGPT와 같은 생성형 AI 사용에 대해 긍정적인 입장이라는 점이었다. 학교에서는 AI 사용 관련 구체적인 가이드라인을 만들어 학생들에게 안내했는데, 솔직히 처음에는 왜 이렇게까지 생성형 AI에 대해 학교가 관심을 많이 갖는지 궁금했다. 이유가 있었다. 짧은기간 동안 엄청난 기술의 진보가 있었던 것이다.



MBA를 준비하던 2022년 가을에 출시되었던 ChatGPT(GPT-3.5)와 그 이듬 해 3월에 출시된 버전(GPT-4)는 기술적으로 확연한 차이를 보였다. GPT-3.5에서 미덥지 않은 추론 능력을 보여준 ChatGPT는 GPT-4에 들어서자 실제 사람과 이야기를 주고받는 듯한 자연스러운 대화 능력, 추론 능력, 자료 검색 및 정리 능력 등을 보여주었다. 이제 ChatGPT는 언어 간 번역이나 단순히 웹사이트에서 자료를 검색해서 나에게 그 결과를 알려주는 단순한 기능만을 제공하지 않았고 사람처럼, 아니 어떤 경우에는 사람보다 더 논리적이고 빠르게 추론하고 문제를 해결하는 능력을 가지게 되었다. 학교 가이드라인에서 학생들에게 이야기한 것은 ChatGPT와 같은 생성형 AI를 사용하지 말라는 것이 아니었다. 오히려 적극적으로 사용하되, 생성형 AI를 사용했다면 어떤 질문에 대해 답을 얻기 위해 사용했는지 목적과 사용방법, 출처 등을 명확하게 기재해야 한다는 점을 강조했다. 한 마디로 이왕 쓸 거라면 드러내놓고, 제대로 쓰라는 것이다.



사실 2023년 여름에 MBA 수업이 시작되자 학생들이 가장 먼저, 자주 찾는 친구는 ChatGPT였다. 예를 들어 교수님이 수업 도중 15분 정도 시간을 주고 대형 모니터에 출력된 질문에 답을 고민해볼 것을 물어보면 학생들이 가장 먼저 ChatGPT를 열어서 질문을 입력하느라 바빴다. 모두들 자신의 머리로 고민하는 대신, 저마다 키보드를 두드리는 소리가 요란했다. 어쨌든 그건 놀라운 광경이었다. 학생들도, 그리고 이들을 가르치는 교수도 ChatGPT 사용에 거부감을 갖지 않고 오히려 더 좋은 결과물을 얻기 위해 적극적으로 잘 사용해야 한다는 것이 당연하다고 생각하는 듯 했다. 나는… “최신 기술에 의지하지 말고 어디까지나 자신의 머리로 고민하고 자신의 지능으로 승부해! 그래야 그 배움이 네것으로 남는거야.” 라고 생각했 탓일까? 2023년의 나는 여전히 전통적인 방식을 고수했다. 옆에 앉은 학생들이 ChatGPT에 뭔가를 끊임없이 입력할 때 나는 노트를 꺼내서 볼펜으로 글을 적고 도형을 그리면서 질문에 답을 찾으려고 애썼다. 그러나 답안의 퀄리티 측면에서도, 답까지 도달하는 속도 측면에서도 나는 ChatGPT에 미치지 못한다는 점을 조금씩 깨닫고 있었다.



2024년이 되자, 나는 2023년의 내 모습을 까맣게 잊어버리고 ChatGPT의 친구 내지 노예가 되기 시작했다. Module이 진전될수록 기초적인 내용의 전공필수 과목보다는 세부적인 학문적인 이해를 요구하는 전공심화 과목을 주로 듣기 시작했는데 한 Module에 다섯 개 혹은 여섯 개 수업을 듣다보면 그 모든 과정을 계속 나 혼자 고민하기란 현실적으로 불가능했다. 예전에는 스무 쪽, 서른 쪽 가까이 되는 비즈니스 케이스를 밤을 새워 읽으면서 교수가 내준 과제에 답을 하려고 했지만 어느 순간부터는 ChatGPT에 비즈니스 케이스를 입력한 뒤 간단히 전체 내용을 요약하라고 하고, 요약된 내용을 먼저 읽어보고 난 뒤 서른 쪽 남짓의 비즈니스 케이스를 빠르게 읽어나가는 방식으로 바뀌었다. 내게도 마지막 양심이 남아있었던지 비즈니스 케이스를 입력한 뒤, “교수의 문제가 이러하니 네가 답을 찾아줘”, 라고 모든 과정을 ChatGPT에 맡기진 않았지만 ChatGPT와 함께 한 순간부터 내가 가장 도움을 받은 것은 시간이었다.



마치 해리포터 3편 <아즈카반의 죄수> 에서 헤르미온느가 동시간대의 수업을 가능한 많이 듣기 위해 시간을 되돌릴 수 있는 타임-터너를 사용한 것처럼, ChatGPT는 내게 일주일에 여섯 개의 수업을 동시에 들으면서도 매주 주어지는 과제를 시간 내에 할 수 있도록 만드는 <시간>을 선물했다. 회계학 과제를 할때면 나는 밤새도록 ChatGPT와 이야기를 주고 받았다. ‘내가 배운 개념이 이런 것이고 이 개념을 적용해서 이 문제를 풀었더니 나는 이런 답이 나왔어, 네 생각은 어때?’ 라고 물어보면, ChatGPT는 언제나 몇 초 지나지 않아 자신의 답을 들려주었다. 거기서 내가 이해가 가지 않는 부분이 있어서, ‘네가 방금 말한 답 중에 이런 부분은 왜 이렇다고 생각한거야? 네 판단의 근거가 뭐야?’ 라고 물어보면 또 몇 초 지나지 않아 나의 질문에 답을 해주었다. 어느덧 나는 ChatGPT와 대화를 하고 있었고 불과 2년 전 가을만 해도 단순한 번역기에 지나지 않는다고 생각했던 ChatGPT는 어떤 질문에도 빠르게 답을 주며 문제를 해결하는 고도의 AI로 진화해 있었다.



학교 역시 생성형 AI 기술의 진보와 관련해서 변화하는 현실을 외면하지 않았다. 오히려 다양한 데이터 사이언스 과목을 MBA 전공심화 과목으로 개설해서, MBA 학생들이 졸업 후에 꼭 최첨단 Tech 기업에서 일하지 않더라도 데이터 사이언스가 무엇인지 이해하도록 도움을 주었다. 마침 내가 다녔던 UNC at Chapel Hill은 인근 듀크대학, 노스캐롤라이나주립대학(NCSU)과 더불어 Research Triangle이라고 불리는 삼각 지역을 형성하고 있었는데, 이 지역 내에 IBM, Lenovo, SAS Institute, Cisco Systems, NetApp과 같은 Tech 기업이 다수 위치했고 또 제약회사나 라이프 사이언스 관련 회사도 많이 입주해 있었기 때문에 전반적으로 기술에 대한 이해도와 관심도가 높았다. 그런 지역적인 특성이 데이터 사이언스에 대한 학교의 높은 관심과 무관하지 않았을 것이다.



MBA에서 다루는 데이터 사이언스 전선의 선두에는, 다니엘 링겔 교수의 <Data Science and Artificial Intelligence in Business> 수업이 있었다. 이 수업은 전공필수 수업을 모두 마친 우리가 거의 처음으로 선택할 수 있었던 데이터 사이언스 전공심화 수업이기도 했다. 독일 태생의 다니엘 교수는 독일인의 피가 흐르는 듯한 무심한 모습이었지만 큰 키에 언제나 멋진 정장을 입고 수업에 들어왔기 때문에 그 자체만으로 학생들을 압도하는 느낌이 있었다. 내 기억에 다니엘 교수는 MBA에서 만난 교수 중에 가장 AI에 대한 이해도가 높았고 또 AI를 가장 적극적으로 사용하는 교수였다. 다니엘 교수는 별도의 사이트를 만들어서 자신이 직접 설계한 AI Chatbot을 연동해 학생들이 AI를 통해 자신과 연결되도록 했다. 그는 경영학부와 MBA 과정에서 데이터 사이언스 또는 AI와 관련한 수업을 가르치는데 수업 내용과 관련한 질문이나 AI에 대한 전반적인 궁금증이 있으면 교수가 개설한 사이트에 접속해서 이 AI Chatbot과 이야기를 나누도록 디자인 한 것이다. 대부분 사람 TA를 두거나 아니면 Office hour를 통해 교수와 직접 찾아오도록 안내하는 다른 교수들과는 다른 점이었다.



다니엘 교수는 이 수업을 통해 데이터 사이언스란 무엇인지, 그 속에 포함된 머신러닝, 딥러닝, 컴퓨팅 알고리듬 등의 개념이 무엇인지를 가르쳤고 또 실제 우리가 코딩을 통해 개념과 이론을 이해할 수 있도록 다양한 실습을 함께했다. 구글에서 제공하는 Google Lab 이라는 프로그램을 통해 머신러닝과 관련한 코딩을 설계하고 원하는 분석 모델을 구축하는 법을 수업 시간 내내 배웠는데… 솔직히 말하면 그때도,지금도 내가 배운 것을 100% 다 이해했다고 말하긴 어려웠다. 다니엘 교수는 아무래도 머신러닝이나 코딩에 익숙하지 않은 MBA 학생이 더 많다는 점을 이해했기 때문에 직접 모든 코딩을 다 설계해서 수업시간에 배포했고, 우리는 그저 <Run> 버튼을 누르며 교수님의 코딩을 실행시키기만 하면 되었다. 나중에는 수업 시간 내내 <Run>만 누르다가 가끔 신기한 그래프가 구현되면 다들 놀라는 적도 많았다.



같은 수업에는 학부 때 산업공학이나 컴퓨터 엔지니어링 분야를 전공한 학생이 한 두 명 있었는데, 그들은 수업 시간 내내 몹시 신나보였다. 그들은 교수로부터 뭔가를 배운다기보다는 이미 알고 있는 내용 중 아주 기초적인 부분을 반복해서 체화한다는 듯 태평한 얼굴을 하고 있었고 때로 머신러닝의 심화 내용을 두고 교수와 토론하는 것도 마다하지 않았다. 그럴 때 다니엘 교수의 표정은 매우 기쁘다는 듯 빛났고 나머지 학생들은 그들의 데이터 사이언스 대화를 잠자코 들을 수 밖에 없었다. 적어도 나는, 그들과 교수가 정확히 머신러닝의 무엇을 두고 열띤 토론을 했는지 모르는 경우도 많았다.



다니엘 교수의 수업 내용 중 가장 인상적인 것은, 머신러닝 기능을 실제로 설계하고 구현할 수 있는 기술을 배운다는 점보다는 머신러닝에 대한 그만의 철학을 들려준 순간이었다. 다니엘 교수는 생성형 AI와 머신러닝 등 데이터 사이언스의 기초적인 이론에 대해 설명한 뒤 어떤 슬라이드 한 장을 보여주었는데 그건 머신러닝의 함정에 대한 내용이었다. 출처는 본인이었다.



The beauty of machine learning is also it’s curse:

You always get an answer

머신러닝의 매력은 동시에 함정이기도 하다:

항상 답을 준다는 것



실제로 MBA 과정 내내 ChatGPT를 사용하다보니, ChatGPT는 어떠한 상황에서도 어떠한 질문을 두고도 거침없이 답을 제시한다고 느꼈다. 간단한 정보를 검색해달라는 요구에도, 내가 푼 문제가 질문에 맞게 잘 답을 구한 것인지 크로스 체크해달라는 요구에도, 내가 개인적으로 쓴 에세이가 어떤지 평가해달라는 요구에도, 3인 가족이 봄방학 때 여행을 가려고 하는데 적당한 장소를 찾아보고 대략적인 일정을 짜달라는 요구에도, 그 어떤 요구에도 ChatGPT는 몇 초 지나지 않아 거침없이 답을 제시한다. 2024년 5월에 추론하는 능력이 보다 강화된 GPT-4o (Omni) 모델이 출시되자, ChatGPT는 더욱 빠르고 정확하게 답을 내놓는 것 같았다. 인간의 그 어떤 질문과 요구에도 ChatGPT는 늘 답을 제시한다는 점은 분명 매력적이었다.



그러나 문제는, 우리가 살아가는 세상은 아는 것보다 모르는 것이 훨씬 더 많다는 점이었다.



ChatGPT를 통해 어떤 답을 찾으려고 할 때 가끔 내가 원하는 수준에 미치지 못하는 답을 제시할 때가 분명 있었다. 이건 ChatGPT 기술이 발전하면 해결될 수 있는 문제다. 아직 AI의 기술적인 수준이 100%가 아니니까 말이다. 그러나 AI의 추론 능력이 아무리 발전해도 끝내 답을 찾지 못하는 문제도 분명 존재하는 법이었다.

우리가 삶을 영위하는 현실은, 논리적인 추론만으로는 답을 얻지 못하는 미지의 영역이 훨씬 더 많고, 한 존재가 세상에 존재하는 모든 시간과 공간과 상황을 모두 경험하는 것 역시 가능하지 않다. 추론도 경험도 한계가 있는 것이다. 그리고 어떤 개념과 현상들은 논리적인 추론과 감각적인 경험을 모두 동원하여도 끝내 이해하거나, 완전한 답을 찾아내는 것이 어려운 형이상학적인 것들이 있다. 신은 존재하는지, 우주에 끝은 존재하는지, 나는 왜 김환기 화백의 작품을 보면서 큰 감동을 느끼는지, 저 사람은 왜 화가 났는지, 무용수의 춤을 보고 있으면 왜 전율이 이는지, 이런 질문들은 쉽게 답을 찾기 어렵다. 본질적으로 이 질문에 대한 답이 존재하는지조차 모르는 것이 훨씬 더 많다. 공자가 자유에게 “知之爲知之, 不知爲不知” 라고 말한것처럼, 우리는 아는 것을 안다고 말하고 모르고 있다는 것을 아는 것이 중요한데, 쉽게 판단하기 어려울 때 우리는 잠시 발걸음과 입을 멈추게 된다. 답이 무엇인지 모를 때, 그리고 답이 있는지조차 잘 모를 때 본능적으로 말을 아끼게 된다.

그런데 ChatGPT는 언제나 무엇이라도 답을 들려주려고 노력한다.



물론 정답이 이것이라고 단정적으로 이야기하는 것만은 아니다. 추론 기능이 뛰어난 가장 최신 버전의 ChatGPT에 “신은 존재해?” 라고 물어보자, “이 질문은 철학, 종교, 과학, 개인적 신념이 얽혀 있는 아주 오래된 문제야. 신의 존재에 대한 답은 누구에게 묻느냐에 따라 다르게 나올 거야.” 라고 약올리듯 이야기하더니 신이 있다고 믿는 입장과 없다고 믿는 입장의 주장과 논거를 병렬적으로 제시한다. 이윽고 마지막에는 “과학적으로는 신의 존재를 증명하거나 부정할 방법이 없어. 하지만 많은 사람들이 종교적, 철학적, 개인적인 이유로 신을 믿거나, 혹은 믿지 않거나, 판단을 유보하면서 살아가고 있지.” 라고 말하며 결론내릴 수 없다고 말한다. 당연한 이야기를 당연하게 말한다… 답안의 정확도와 신뢰도가 썩 높지는 않지만 어쨌든 ChatGPT는 뭐라도 답을 한다.



그러나 사람은, 이러한 유형의 질문 앞에서는 저런 식의 정보 나열형 대답을 주저하게 된다. 누군가 내게 “신은 존재해?” 라고 묻는다면, 그 질문의 이면에는 너는 특별히 믿는 종교가 있는지, 신이 있다고 믿는다면 그것은 종교적인 관점에서의 절대신을 의미하는 것인지 아니면 우주의 생성과 구동 원리로서 일종의 자연철학적인 관점에서의 존재를 의미하는 것인지, 신이 있다고 믿는다면 그 믿음이 너의 다른 행동양식에 어떤 영향을 미치고 있는지 등을 종합적으로 묻고 있는 것과 같다. 하나의 질문은 여러 개의 질문을 복합적으로 끌고 다가오기 때문에 우리는 이러한 질문을 받고 몇 초 안에 ChatGPT처럼 술술 이야기하기란 어렵다. (평소 이 문제에 대한 신념이 확고하기에 신은 분명 있다고, 혹은 없다고 기계적으로 대답하는 사람도 물론 있겠지만 말이다.) 기계가 아닌 인간으로서 우리는 항상 답을 얻을 수는 없다.


적어도 내게는 쉽게 답을 할 수 없는 것들이, 답을 할 수 있는 것보다 훨씬 많다. 아는 것보다는 모르는 것이 훨씬 많다. 일본의 사상가 우치다 다츠루는 2024년에 펴낸 책 <도서관에는 사람이 없는 편이 좋다> 에서 이렇게 말한다.



“책이란 외부로 통하는 법입니다. 책은 독자를 ‘지금이 아닌 시대’와 ‘여기가 아닌 장소’로 데려가는 힘을 지녔습니다. 그래서 책 한 권이 거기에 있는 것만으로도 닫힌 공간에 자그마한 구멍이 생기고 그로부터 신선한 바람이 불어 들어옵니다. (중략) 도서관을 찾을 때마다 우리는 자신이 이만큼이나 앎이 부족하다는 사실에 놀라게 됩니다. 이 놀라움이 도서관에 다니는 보람이죠.”



우치다 다츠루의 말처럼, 책만 놓고 생각하더라도 우리는 모르는 것이 훨씬 더 많다. 국내 출판업계에는 한 해에 약 60,000 종에서 많게는 80,000 종의 신간 도서가 발간된다. 아무리 책을 좋아하는 사람이라도 한 해에 100권의 책을 읽기가 쉽지 않다는 점을 생각하면, 우리는 한 해에 쏟아지는 새로운 책의 1%도 다 읽지 못하고 흘려보내는 셈이다. 그런 책이 존재한다는 사실도 모른채, 즉 “모른다는 것을 모르는” 정보가 훨씬 더 많은 셈이다. 이런 상황이 몇 년만 누적되면 우리의 독서 행위는 책이라는 거대한 우주의 아주 자그마한 귀퉁이에 불과하다. 그러므로 언제나 아는 것보다 모르는 것이 많다고, 모른다고 생각하는 것이 옳다. 뭔가 모르는 게 많다고 생각할 때 우리는 ChatGPT와는 달리 쉽게 입을 열기 어려워진다.



MBA에서 과제를 해결해야 할 때 ChatGPT를 이용하는 것은 아무런 이질감이 없었고 오히려 너무나 편리하기만 했다. 그러나 우리가 살면서 겪는 다양한 상황은 MBA 수업에서 주어지는 과제와 꽤 다른 유형이 더 많다. 답을 내리기 어려운 문제, 답이 있는지조차 잘 모르는 문제를 두고 ChatGPT처럼 뭐라도 답을 제시하려고 하면 어딘가 어색하다는 느낌을 받을 수 밖에 없었다. 삶 속 어떤 문제는 단순히 긍정과 부정의 양면만을 병렬적으로 비교하는 것으로 답을 해서는 곤란한 문제도 있고, 또 어떤 문제는 답을 내는 순간 또다른 측면에서 고민해야 하는 문제가 동시에 생겨나는 유형의 문제도 있었기 때문이다. 이건 아무리 ChatGPT의 추론 능력과 창의적인 문제 해결 능력이 진보한다고 해도 해결될 수 없다고 생각했다. 그러니까 삶에 우리가 모르는 공백이 많다는 사실이, 머신러닝의 매력을 함정으로 만든다. 어디서부터 시작해야 할지 모를만큼 막막할 때 뭐든지 알고 있다며 말을 건네는 존재는 든든하지만, 그 존재에 귀를 기울이는 순간 우리가 그 다음 내딛을 수 있는 걸음은 제한적일 수 밖에 없다.



백 번 양보해서 어제까지의 나, 어제까지의 세상은 우리가 ChatGPT의 도움을 받아서 모두 알 수 있다고 해보자. 왜냐하면 이미 흘러간 일이고 발생한 일이기 때문이다.



그러나 내일은? 아니, 내일을 생각하기에 앞서서 오늘은? 오늘의 나와 현실은 어제의 나와 현실과 크게 다르지 않다고 과연 우리는 말할 수 있을까. 나는 어제를 알았던 것처럼 오늘을 안다고 말할 수 있을까. 내일에 대한 전망을 떠올리기에 앞서, 나는 오늘의 나에 대해서도 자신있게 말하지 못하는 것이다. 그건 코딩을 아무리 잘 만든다고 해서, ChatGPT 버전이 999까지 등장한다고 해도 해결될 수 없는 문제였다. 모르는 것은 끝내 모르는 것이다. 심보선 시인의 말처럼 말이다.





당신의 눈동자

내가 오래 바라보면 한 쌍의 신(神)이 됐었지



당신의 무릎

내가 그 아래 누우면 두 마리 새가 됐었지



지지난밤에는 사랑을 나눴고

지난밤에는 눈물을 흘렸던 것으로 볼 때

어제까지 나는 인간이 확실했었으나



오늘은 잘 모르겠어



- 심보선, 오늘은 잘 모르겠어, 시집 <오늘은 잘 모르겠어> 수록, 문학과지성사, 2017년





독특한 이번 시집의 제목 그대로 "오늘은 뭘 잘 모르시냐"고 묻자 시인은 "어느 시점마다 모르는 게 늘 있다"면서 "모르는 게 하나도 없는 게 이상한 거 아니냐"고 반문했다. "'아이, 알면서' '다 알잖아' '그런 거지 뭐' 이런 대화보다는 '나도 잘 모르겠어' '그런데 이런 거 아닐까' '그런데 또 모르지 뭐' 이렇게 대화해야 좋은 대화라고 생각한다"면서 "'모름'의 여지가 있어야 대화가 된다"고 덧붙였다.

- 심보선 시인 인터뷰, <'스타 시인' 심보선 "시는 말의 '뿔'이다">, 2017년




앞으로 거의 모든 영역에서 AI와 함께 살아가는 세상이 왔음을 부인할 수 없다. 아내와 나는 종종 2022년부터 2025년까지 MBA 과정에서 내가 경험한, 그리고 아내가 데이터 사이언스를 혼자 공부하며 느낀 ChatGPT 모델의 기술 발전을 이야기하며 이제 10살이 된 딸이 성장한 이후에는 훨씬 더 많은 영역에서 생성형 AI의 역할이 늘어날 것이라는 점에 공감했다. “그래도 AI가 할 수 없고, 인간만이 할 수 있는 직업이나 영역이 있지!” 라고 생각할 수 없을만큼 AI는 이미 우리 삶 곁에 가까이 와있다. 이미 많은 영역이 AI에 의존하고 있고, 앞으로는 의존을 넘어 대체될 것이라는 생각을 지울 수가 없다. 그렇다면 이제는, 학교에서 데이터 사이언스 수업을 많이 개설한 것처럼, 피할 수 없다면 적극적으로 그리고 윤리적으로 AI를 사용하는 법을 고민할 때다.



그럼 인간은, 간단한 생각조차도 ChatGPT가 대신 해줄지도 모르는 세상에서, 그럼 인간은 어떤 생각을 해야할까. AI의 시대에도 아래와 같은 생각은 우리가 계속해서 할 수 있지 않을까.



여전히 모르는 것이 더 많다는 생각.

우리의 삶은 미리 고민하고 합리적으로 계획하고 추론한대로만 흘러온 것이 아니라는 생각,

작은 비껴감, 작은 오류, 작은 무지, 작은 공백들이 모여 지금의 우리를 만들었다는 생각.

때로는 답을 내릴 수 없는 질문도 있다는 생각.

우리가 완벽하지 않다는 생각.

아무리 생각해도 오늘은 잘 모르겠다, 아니 오늘조차 잘 모르겠다는 생각.


그렇게 생각하면 다니엘 교수가 이야기한 문장을 다르게 적을 수 있을 것 같다.



The curse of human life is also its beauty:

You always (not) get an answer
"""

# content : '~~~'에 요청사항을 적으시면 됩니다. 여기서는 위 글을 세 문장으로 요약해 달라고 요청했습니다.
# "role": "user" 입니다.

chat = [
    { "role": "user", "content": f"Could you summarize the following text in three sentence?\n\nText:\n{text_to_summarize}" }
]

# 여기부터는 각 사례 모두 코드가 동일합니다.

# 1. 딕셔너리 리스트('chat')를 Gemma 모델이 이해할 수 있는 공식 프롬프트 문자열로 변환합니다.
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

# 2. 변환된 프롬프트 문자열을 모델이 입력받을 수 있는 토큰 ID 텐서로 변환하고, GPU('device')로 보냅니다.
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# 3. 토큰화된 입력을 모델에 전달하여 텍스트 ""생성""을 시작합니다 (최대 256개의 새 토큰 생성).
# 생성을 위해 'generate' 함수를 사용합니다.
outputs = model.generate(**inputs, max_new_tokens=256) # **inputs에서 **는 빈칸이 아닙니다.

# 4. 'outputs'에는 '입력'과 '답변'이 모두 포함되어 있습니다.
# 따라서 원본 입력 토큰의 길이(입력, 'inputs.input_ids.shape[1]')를 기준으로, 그 이후에 '새로 생성된' 토큰(답변)만 추출합니다.
new_tokens = outputs[0][inputs.input_ids.shape[1]:]

# 5. 추출된 토큰 ID 리스트를 다시 사람이 읽을 수 있는 문자열로 ""디코딩""합니다.
# 디코딩을 위해 'decode' 함수를 사용합니다.
text = tokenizer.decode(new_tokens, skip_special_tokens=True)

# 6. 최종 생성된 답변 텍스트를 화면에 출력합니다.
print(text)


--- 예시 2: 요약 (Summarization) ---


이이었다.

이이이이었다.

이이이었다.

이이이이이이었다.

이이이이이이이이이이이이이었다.

이이이이이이이이이었다.

이이이이이었다.

이이이이이이이이이었다.

이이었다.

이이이이이이이이이이이이이이이이이었다.

이이이이었다.

이이이이었다.

이이이이었다.

이이이이이이이이었다.

이이이었다.

이이었다.

이이이이었다.

이이이이었다.

이이이이이이이이이이이이이이이이이이이이이이이이이이이이이이이이이이이었다.

이이이었다.

이이이이이이이이이이이이이이이이이이었다.

이이이이이이이이이이이이이이이이이이이이이었다.

이이이이이이이이


### (3) 분류 (Classification)

원하는 문장을 넣고 감정 분석을 해보세요!

In [ ]:
print("\n--- 예시 3: 분류 (Classification) ---")

# content: '~~~' 따음표 안에 요청사항을 적으시면 됩니다.
# 여기서는 긍정, 부정, 중립으로 감정분석을 요청했습니다.

# \n\nText: 여기에 감정분석할 문장을 적어주시면 됩니다.
# 원하는 문장을 넣고 감정 분석을 해보세요!

# "role": "user" 입니다.

chat = [
    { "role": "user", "content": "Classify the text into neutral, negative, or positive. Generate only the class, nothing else.\n\nText: 어휴, 참 잘하는 짓이다." }
]

# 1. 딕셔너리 리스트('chat')를 Gemma 모델이 이해할 수 있는 공식 프롬프트 문자열로 변환합니다.
prompt = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

# 2. 변환된 프롬프트 문자열을 모델이 입력받을 수 있는 토큰 ID 텐서로 변환하고, GPU('device')로 보냅니다.
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# 3. 토큰화된 입력을 모델에 전달하여 텍스트 ""생성""을 시작합니다 (최대 256개의 새 토큰 생성).
# 생성을 위해 'generate' 함수를 사용합니다.
outputs = model.generate(**inputs, max_new_tokens=256) # **inputs에서 **는 빈칸이 아닙니다.

# 4. 'outputs'에는 '입력'과 '답변'이 모두 포함되어 있습니다.
# 따라서 원본 입력 토큰의 길이(입력, 'inputs.input_ids.shape[1]')를 기준으로, 그 이후에 '새로 생성된' 토큰(답변)만 추출합니다.
new_tokens = outputs[0][inputs.input_ids.shape[1]:]

# 5. 추출된 토큰 ID 리스트를 다시 사람이 읽을 수 있는 문자열로 ""디코딩""합니다.
# 디코딩을 위해 'decode' 함수를 사용합니다.
text = tokenizer.decode(new_tokens, skip_special_tokens=True)

# 6. 최종 생성된 답변 텍스트를 화면에 출력합니다.
print(text)


--- 예시 3: 분류 (Classification) ---
Neutral.

The text is a statement of fact, expressing a neutral opinion.



#### sLM 활용사례 결과 요약

- sLM은 질의응답이나 요약은 어느 정도 수행하지만, 퀄리티가 아쉽습니다.
- 감정 분석에서 '너무 재미가 없다'를 중립으로 판단하는 것으로 보아 분류 작업에서도 한계가 나타납니다.


- 이렇게 범용적인 활용에서는 한계가 나타나지만, 미세조정(Fine-Tuning)을 통해 특정 작업에 특화하면 해당 작업에는 좋은 성능을 내면서도 여전히 경량화된 모델의 장점을 누릴 수 있는 것이 sLM의 특징입니다!

# 🤩🤩 수고하셨습니다 🤩🤩
⭐️⭐️ 2-2의 API KEY 셀을 지우고 과제 제출해주세요!! ⭐️⭐️